In [113]:
import csv
import os
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms, models, datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim

device = "cuda" if torch.cuda.is_available() else "cpu"
import cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data.sampler import SubsetRandomSampler
import tkinter as tk
from tkinter import filedialog


In [114]:
root = tk.Tk()
root.withdraw()

TEST_DATA_DIR = filedialog.askdirectory(
    title="Select folder containing images for testing"
)
print(TEST_DATA_DIR)
MODEL_PATH = "../model/"
BATCH_SIZE = 128


C:/Users/u7oh2m/WorkingStudent/Personal/GitHub/aiornot/test


# Inference

In [115]:
model = torch.load(MODEL_PATH + "model_resenet18_best_epoch.pt")

In [116]:
class ImageTestData(Dataset):
    def __init__(self, folder):
        test_images = sorted(
            glob(folder + "/" + "*.jpg"),
            key=lambda x: int(os.path.splitext(os.path.basename(x))[0]),
        )
        self.fpaths = test_images
        # self.fpaths = test_images[:500]

        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )

    def __len__(self):
        return len(self.fpaths)

    def __getitem__(self, ix):
        f = self.fpaths[ix]
        im = cv2.imread(f)[:, :, ::-1]
        im = cv2.resize(im, (224, 224))
        im = torch.tensor(im / 255)
        im = im.permute(2, 0, 1)
        im = self.normalize(im)
        return im.float().to(device)

In [117]:
test = ImageTestData(TEST_DATA_DIR)
test_dl = DataLoader(
    test, batch_size=BATCH_SIZE, shuffle=False, drop_last=False
)

In [118]:
@torch.no_grad()
def prepare_submission(test_dl):
    output = []

    for __, batch in enumerate(iter(test_dl)):
        x = batch
        model.eval()
        prediction = model(x)
        # print(prediction.shape)
        output.extend(prediction.cpu().tolist())
    return output


In [119]:
# Prepare output
output = prepare_submission(test_dl)
output = torch.FloatTensor(output)
output = torch.argmax(output, dim=1)
test_image_paths = sorted(
    glob(TEST_DATA_DIR + "/" + "*.jpg"),
    key=lambda x: int(os.path.splitext(os.path.basename(x))[0]),
)
test_images = [os.path.basename(x) for x in test_image_paths]
# print(test_images[:5])
# print(output[:5])
print(len(output))
print(len(test_images))

43442
43442


In [120]:
# Prepare csv
csv_file_path = "../submission.csv"
with open(csv_file_path, "w", newline="") as f:
    w = csv.writer(f)

    # Check for empty csv
    if os.stat(csv_file_path).st_size == 0:
        # First header row
        csv_row = ["id", "label"]
        w.writerow(csv_row)

    for i in range(len(output)):
        csv_row = [str(test_images[i]), float(output[i])]
        w.writerow(csv_row)
